# Preziceri pentru Clienti Pas cu Pas:


# 1. Import Data

In [3]:
import pandas as pd
customers_data = pd.read_csv('customers.csv')
customers_data

,customer_name,pickup_city,dropoff_city,total_income
0,Timi Customer,New York,San Antonio,500
1,Larisa Customer Co,Los Angeles,San Diego,1000
2,Gabi Customer,Chicago,Texas,1500
3,Raul Customer,Dallas,San Jose,2000
4,Elias Customer,Mexico City,Austin,2500


# 2. Clean the Data 
- Pentru acest tip de data nu este necaser sa Curatam Datele.
- Dar trebuie sa impartim datele in 2 seturi de date separate !
## Datele de intrare vor fi toate coloanele mai putin ultima coloana. 
## Datele de iesire (predictiile) va fi ultima coloana.



In [4]:
X = customers_data.iloc[:, :-1].values   # Datele de intrare
X

array([['Timi Customer', 'New York', 'San Antonio'],
       ['Larisa Customer Co', 'Los Angeles', 'San Diego'],
       ['Gabi Customer', 'Chicago', 'Texas'],
       ['Raul Customer', 'Dallas', 'San Jose'],
       ['Elias Customer', 'Mexico City', 'Austin']], dtype=object)

In [5]:
y = customers_data.iloc[:, -1].values    # Datele de iesire
y

array([ 500, 1000, 1500, 2000, 2500], dtype=int64)

## 2.1 Encoding categorical Data
## Putem aplica 2 metode populare:
<hr>
(Label Encoding): Dacă datele de ieșire reprezintă clase sau categorii ordonate, poți folosi codificarea etichetelor. Acest lucru presupune transformarea fiecărei etichete într-un număr întreg, astfel încât să poată fi utilizată de algoritmi. De exemplu, dacă ai clasele "Cățel", "Pisică" și "Șoarece", le-ai putea transforma în 0, 1 și 2.

(Acesta metoda este utila atunci cand avem date ordonate cum ar fi chiar si datele de intrare de tip ['S', 'M', 'L'] care reprezinta dimensiunile unor haine, sau 'YES', 'NO' care reprezinta raspunsuri la intrebari)

<hr>
(One-Hot Encoding): Dacă datele de ieșire reprezintă clase sau categorii care nu sunt ordonate, poți folosi codificarea one-hot. Această metodă transformă fiecare clasă într-un vector binar, în care doar una dintre componentele este 1, corespunzătoare clasei respective. De exemplu, dacă ai clasele "Roșu", "Verde" și "Albastru", transformarea ar fi [1, 0, 0], [0, 1, 0] și [0, 0, 1], respectiv.
<hr>


## > Encoding the Independent Variable (datele de intrare vor fi converite) <
## Acest pas este un pas importat in pregatirea datelor deoarece asigura ca modelul invata corect!
### Modelele de ML lucreaza intotdeaua cu DATE NUMERICE avand date de tip categoric (customer_name, pickup_city, dropoff_city). Pentru a face predicti cat mai bune si precise avem nevoie sa codificam datele categorice (str) in date numerice (int).
### Una din cele mai comune metode este ONE-HOT ENCODING (acesta metoda converteste valoriile categorice intr-o coloana noua binara care contine un set de numere binare cu 0 si 1.
Pentru datele noastre din ['customer_name'] vom transforma acesta coloana in 5 coloane categorice deoarece avem 5 nume diferite.

Exemplu:
Avem o coloana ['Culoare'] in care avem valoriile ['Rosu', 'Verde', 'Albastru']. După codificarea one-hot, această coloană va fi divizată în trei coloane separate: "Culoare_Rosu" [1 0 0], "Culoare_Verde" [0 1 0] și "Culoare_Albastru" [0 0 1].

Importarea modulelor:
- OneHotEncoder o folosim pentru a aplica metoda ONE-HOT ENCODING
- ColumnTransformer este o clasa pe care o folosim pentru aplicarea transformari specifice ale coloanelor.

Parametrii ColumnTransformer:
- transformers= parametru cheie care reprezinta o lista de tupluri care definesc transf. aplicate.
- - 'encoder' nume pentru transformare
- - OneHotEncoder() este obiectul transformatorului
- -  [0, 1, 2] lista de index ale coloanelor pe care se aplica transformarea
- remainder= acest parametru specifica ce se intampla cu coloanele care nu sunt afectate de transformers, 'passthrough' pastrarea lor nemodificate

## > Encoding the Dependent Variable (datele de iesire vor fi convertite) <
## Daca aveam si datele de iesire de tip str trebuiau convertite


In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2])
                                     ],
                       remainder='passthrough'
                       )
# X = np.array(   # convertim OBJ intr-un array Numpy deoarece avem nevoie de o matrice Numpy pt a instrui modelele
#     ct.fit_transform(X)     # aplica transformările specificate în transformator la setul de date X
# )

X = ct.fit_transform(X)     # aplica transformările specificate în transformator la setul de date X
X = X.toarray()   # convertim matricea rară într-un array numpy dens
X

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.]])

# 3. Split the Data into Training/Test Sets
## Vom desparti datele in 2 seturi de date: un set de date de antrenament si un set de date de testare.
### Setul de date de antrenament va fi folosit pentru a antrena modelul.
### Setul de date de testare va fi folosit pentru a testa cât de bine funcționează modelul pe date noi, pe care nu le-a văzut în timpul antrenamentului (new observation). Aceste new observation sunt datele pe care modelul nu le-a vazut in timpul antrenamentului si pe care le va folosi pentru a face predictii.

- Impartim datele de intrare X in 2 seturi de date: X_train si X_test (80% din date vor fi pentru antrenament si 20% pentru testare), acelasi lucru il facem si pentru datele de iesire y.

In [7]:
from sklearn.model_selection import train_test_split

# functia va returna o lista de tuple, fiecare tuplu contine 2 elemente: X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=1)   # test_size=0.2 reprezinta 20% din date pentru testare

Deciziile luate pentru cei 4 Customers din y_train ii corespund pentru cei 4 Customers din X_train. 80%

Deciziile luate pentru 1 Customers din y_test ii corespund pentru cel 1 Customers din X_test. 20%  Aceste date le vom obtine mai tarziu din productie.

In [8]:
X_train

array([[0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [9]:
y_train

array([1000, 2500,  500, 2000], dtype=int64)

In [10]:
X_test

array([[0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [11]:
y_test

array([1500], dtype=int64)

# 3.1 Feature Scaling
## Feature Scaling este un pas important în pregătirea datelor, deoarece modelele de ML se bazează pe distanțe între puncte pentru a face predicții. Dacă datele nu sunt scalate, caracteristicile cu valori mai mari vor avea un impact mai mare asupra modelului decât caracteristicile cu valori mai mici. Acest lucru poate duce la rezultate incorecte.
## Anumite modele de ML, cum ar fi regresia liniară, regresia logistică și k-NN, necesită scalarea caracteristicilor pentru a funcționa corect. Dar pentru DecizionTreeClassifier nu este necesar. 

# 4. Create a Model (DecizionTreeClassifier algoritm supervizat pentru clasificari)
## Acest algoritm ia decizii bazat pe datele de intrare si le clasifica intr-una sau mai multe clase.
### Explicatie din viata reala: E ca un detectiv care incerca sa rezolva un caz. Are un set de indicii (datele de intrare) despre cazul pe care il investigheaza si incearca sa decida cine este vinovatul.
### Acest algoritm construieste ca un arbore plin cu indicii care reprezinta o intrebare despre caracteristicile datelor. In functie de raspuns el se deplaseaza prin arbore  si incearca sa ia o decizie.

Un alt exemplu ar fi:
- De exemplu, să spunem că aveți un set de date cu caracteristici despre fructe (cum ar fi culoarea, forma, dimensiunea etc.) și doriți să clasificați fiecare fruct într-una din trei clase: măr, portocaliu sau banană. DecizionTreeClassifier ar putea începe prin a pune întrebarea: "Este fructul roșu?" Dacă răspunsul este "da", ar putea decide că este un măr. Dacă răspunsul este "nu", ar putea pune o altă întrebare, cum ar fi "Este fructul rotund?" și așa mai departe, până când ajunge la o clasificare finală pentru fiecare fruct.

### În esență, DecizionTreeClassifier este un instrument puternic pentru clasificare, deoarece poate învăța din datele de antrenament și poate face preziceri precise pe baza acestora. 

Pasi:
- Creem o instanta a modelului

In [12]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

# 5. Train the Model
##  Antrenarea modelului înseamnă să îi oferi datele de antrenament și să îi permiți să învețe de la aceste date. În esență, modelul va încerca să înțeleagă relațiile dintre datele de intrare și datele de ieșire, astfel încât să poată face predicții precise pe date noi.

In [13]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

# 6. Make Predictions
## După ce modelul a fost antrenat, putem folosi metoda predict() pentru a face predicții pe date noi. Aceste date noi pot fi date pe care modelul nu le-a văzut în timpul antrenamentului. Ii oferim X_test (datele de intrare) și modelul va returna predicțiile pentru aceste date.

Nu ii furnizam y_test (datele de ieșire) deoarece acestea sunt valorile pe care modelul ar trebui să le prezică daca i le-am oferi i-am oferi direct raspunsul, aste ne-ar afecta calcularea acuratetii modelului.

In [14]:
predictions = model.predict(X_test)
predictions  

array([2500], dtype=int64)

# 6.1 Accuracy
## Pentru a evalua cât de bine funcționează modelul, putem calcula acuratețea acestuia. Acuratețea este o măsură a cât de bine modelul face predicții corecte pe datele de testare.

## Ii oferim y_test (valorile reale) si predictions (valorile prezise de model) si metoda accuracy_score() va returna acuratețea modelului.

- y_test: reprezinta raspunsul corect pentru datele din preziceri 
- predictions: reprezinta valoriile prezise de model

In [15]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
accuracy

0.0

# 7. Evaluate the Model
## Pentru a evalua cât de bine funcționează modelul, putem compara predicțiile modelului cu valorile reale din setul de date de testare. Acest lucru ne va oferi o idee despre cât de precis este modelul în predicția datelor noi.

In [16]:
raspuns_model = predictions
raspuns_corect = y_test

raspuns = pd.DataFrame({'Raspuns Corect': raspuns_corect, 'Raspuns Model': raspuns_model})
raspuns

,Raspuns Corect,Raspuns Model
0,1500,2500


# !!! Persisting Models !!!

## O data ce am antrenat modelul nostru, il vom salva intr-un file astfel incat data viitoare cand dormi sa facem predictii pur si simplu incarcam modelul si facem prezicerii.
## Modelul fiind deja antrenat nu mai trebuie sa il reantrenam inca o data.

Indicat e sa salvam modelul fara preziceri, deoarece prezicerile sunt doar pentru datele de testare si nu pentru datele noi.

Functia _joblib este o functie din sklearn care ne ajuta sa salvam modelul intr-un fisier.

In [19]:
from sklearn.utils import _joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pandas as pd


customers_data = pd.read_csv('customers.csv')

X = customers_data.iloc[:, :-1].values
y = customers_data.iloc[:, -1].values

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2])],
                       remainder='passthrough'
                       )

X = ct.fit_transform(X) 
X = X.toarray() 

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=1)   

model = DecisionTreeClassifier()
model.fit(X_train, y_train)



_joblib.dump(model, 'model.joblib')


['model.joblib']

## Incarcam modelul salvat si facem predictii pentru datele de testare.
## Astfel acum putem face predictii cu un model deja antrenat fara a mai antrena modelul inca o data.

In [26]:
model = _joblib.load('model.joblib')
prediction_model = model.predict(X_test)
prediction_model

array([1000], dtype=int64)